## This notebook clears following concepts
- Basic Structure operations

In [30]:
import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.types import StructField, StructType, StringType, LongType
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.appName("spark_definitive_guide").getOrCreate()

22/10/04 21:37:49 WARN Utils: Your hostname, AS-MAC-0006.local resolves to a loopback address: 127.0.0.1; using fd01:db8:1111:0:0:0:0:3 instead (on interface lo0)
22/10/04 21:37:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/04 21:37:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
manualSchema = StructType([
    StructField('DEST_COUNTRY_NAME', StringType(), True), 
    StructField('ORIGIN_COUNTRY_NAME', StringType(), True), 
    StructField('count', LongType(), False, metadata={"hello": "world"})
])

In [11]:
path = "./../src/flight-data/json/2015-summary.json"
df = spark.read.format('json').schema(manualSchema).load(path)

In [16]:
# constants

df.schema
df.columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

### difference between select and selectExpr
- `select(*cols: ColumnOrName) -> DataFrame`
- `selectExpr(*expr: str) -> DataFrame`

In [34]:
# select and selectExpr

df.select('DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count')
df.select(expr("DEST_COUNTRY_NAME as dcount"))
df.selectExpr("*", "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as sameCountry") # using multiple exprs
df.selectExpr('avg(count)', 'count(distinct(DEST_COUNTRY_NAME))') # no need of import of count/avg functions here


dfWithLongColName = df.withColumn("This colName have spaces", expr("count"))
dfWithLongColName.select("This colName have spaces") # back tick not required and only string name is expected
dfWithLongColName.selectExpr("`This colName have spaces` * 10") # back tick required to separate string name and expression


DataFrame[(This colName have spaces * 10): bigint]

In [37]:
df.withColumnRenamed("DEST_COUNTRY_NAME", "new_name_dest_country") # all cols with one col renamed
df.withColumnRenamed("DEST_COUNTRY_NAME" , "dest_new_name").withColumnRenamed("ORIGIN_COUNTRY_NAME", "orig_new_name")

DataFrame[dest_new_name: string, orig_new_name: string, count: bigint]

In [42]:
df.drop('DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME') # returns another df, dont remove inplace

DataFrame[count: bigint]

In [45]:
df.filter(col("count") < 5)
df.where("count < 5")

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]